In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random as rnd
import math as m
import pandas as pd
import scipy.stats as stat
import scipy.optimize as opt
import seaborn as sb
from scipy.integrate import quad
from scipy.integrate import dblquad
from scipy.special import comb

#plt.style.use('ggplot')

my_norm = lambda exp, disp: stat.norm(loc = exp, scale = m.sqrt(disp))

In [2]:
df = pd.read_csv('Priem.csv', sep = ';', encoding = 'windows-1251')

Как известно,

$$
2 \sum\limits_{i = 1}^{12} \nu_i \ln\left( \frac{12\nu_i}{n}\right) \stackrel{d}{\to} \chi^2_{11}
$$

$$
11 = 12 - 1 - ddof \quad \mathrm{for \ monthes}
$$

для дней аналогично с параметром 31

In [16]:
dates = df['Дата.рождения']

list_monthes = [int(elem[3] + elem[4]) for elem in dates]
freq_monthes = [list_monthes.count(i) for i in range(1, 13)] 
freq_monthes

[121, 97, 107, 106, 132, 124, 116, 127, 132, 117, 103, 108]

In [42]:
month_p1 = round(stat.chisquare(freq_monthes, ddof = 0).pvalue, 4)

In [40]:
list_days = [int(elem[0] + elem[1]) for elem in dates]
freq_days = [list_days.count(i) for i in range(1, 32)] 

days_p1 = round(stat.chisquare(freq_days, ddof = 0).pvalue, 4)

In [48]:
print('  Days\' p-value =  ' + str(days_p1))
print('Monthes\' p-value = ' + str(month_p1))

  Days' p-value =  0.0979
Monthes' p-value = 0.298


Месяцы года встречаются равновероятно.
Дни месяца - то же (если наш уровень значимости не 0.1).

Гипотезы не отвергнуты 

###### Но что если сделать поправку на разное число дней в месяце???

По-прежнему имеет место сходимость к распределению хи-квадрат со степенями свободы 11 или 31, ddof $= 0$

In [68]:
expected_days = [12/365 for i in range(28)]
expected_days.append(11/365)
expected_days.append(11/365)
expected_days.append(7/365)

len_dates = len(dates)
days_p2 = stat.chisquare(freq_days, [len_dates * elem for elem in expected_days ], ddof = 0).pvalue

In [69]:
expected_monthes = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
expected_monthes = [elem / 365 for elem in expected_monthes]

monthes_p2 = stat.chisquare(freq_monthes, [len_dates * elem for elem in expected_monthes ], ddof = 0).pvalue

In [72]:
print('  Days\' p-value =  ' + str(round(days_p2, 4)))
print('Monthes\' p-value = ' + str(round(monthes_p2, 4)))

  Days' p-value =  0.5821
Monthes' p-value = 0.4861


Месяцы года встречаются равновероятно.
Дни месяца - то же.

Гипотезы не отвергнуты.

Поправка заметно увеличила p-value, для дней - сильнее

##### Но что если учесть, что 2000 (и прочие) года високосные?????
уже считать не хочется...

In [83]:
list_years = [int(elem[-2] + elem[-1]) for elem in dates]
freq_years = [list_years.count(i) for i in range(0, 100)] 

leap_counter = 0;
for i in range(len(freq_years)):
    if (i % 4 == 0):
        leap_counter += freq_years[i];

print("Число абитуриентов, родившихся в високосные годы равно", leap_counter)

Число абитуриентов, родившихся в високосные годы равно 288


In [9]:
#какой-то пример
freq_days = [0 for i in range(30)]
freq_days.append(1000)
stat.chisquare(freq_days, [1/31 for i in range(31)], ddof = 0)

Power_divergenceResult(statistic=30998001.000000004, pvalue=0.0)

In [33]:
#какой-то пример
freq_days = [1 for i in range(31)]
stat.chisquare(freq_days, [1 for i in range(31)], ddof = 0)

Power_divergenceResult(statistic=0.0, pvalue=1.0)